# 📚 Transcripción Automática de Documentos Históricos

Este notebook transcribe documentos históricos del siglo XIX usando OCR y los limpia con IA.

**Proceso:** PDF → OCR → Limpieza con Gemini → Documento Word


### Requisitos previos:
- Python 3.11+
- Tesseract OCR instalado
- Poppler instalado
- Librerías Python: `pytesseract`, `pdf2image`, `Pillow`, `tqdm`



## 📦 Paso 1: Instalación de Dependencias

Esta celda instala todas las librerías necesarias para el proceso de transcripción OCR.


In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following yanked versions: 1.13.0
ERROR: Could not find a version that satisfies the requirement pdf2image>=3.1.0 (from versions: 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.1.7, 0.1.9, 0.1.10, 0.1.11, 0.1.12, 0.1.13, 0.1.14, 1.0.0, 1.1.0, 1.2.0, 1.2.1, 1.3.0, 1.3.1, 1.4.0, 1.4.1, 1.4.2, 1.5.0, 1.5.1, 1.5.2, 1.5.3, 1.5.4, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.9.0, 1.10.0, 1.11.0, 1.12.1, 1.13.1, 1.14.0, 1.15.0, 1.15.1, 1.16.0, 1.16.2, 1.16.3, 1.17.0)
ERROR: No matching distribution found for pdf2image>=3.1.0


## 🔧 Paso 2: Código de Transcripción OCR

Esta celda contiene la función principal que:
- Convierte PDF a imágenes de alta calidad
- Aplica OCR con configuración optimizada para documentos históricos
- Procesa página por página con barra de progreso
- Genera archivo Markdown con el texto transcrito


In [4]:
%pip install pytesseract pdf2image Pillow tqdm google-generativeai python-docx

  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   --

In [ ]:
# ==============================================================================
# CÓDIGO DE TRANSCRIPCIÓN PARA JUPYTER NOTEBOOK LOCAL
# ==============================================================================
import pdf2image
from pdf2image import convert_from_path
import pytesseract
import os
from tqdm import tqdm
import platform

# Configuración específica para Windows
if platform.system() == "Windows":
    # Configurar rutas de Tesseract y Poppler para Windows
    # Ajusta estas rutas según tu instalación
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    
    # Configurar Poppler para Windows (ajusta la ruta según tu instalación)
    poppler_path = r'C:\poppler\bin'  # Cambia esta ruta si es diferente
    if os.path.exists(poppler_path):
        os.environ['PATH'] = poppler_path + os.pathsep + os.environ['PATH']

def transcribir_documento_simple(pdf_path, output_path, pagina_inicio=1, pagina_fin=None):
    """
    Versión simplificada sin configuraciones complejas que causan errores
    Optimizada para documentos históricos del siglo XIX
    """
    print("🔄 Convirtiendo PDF a imágenes...")

    try:
        # Para documentos históricos, usar DPI más alto para mejor calidad
        paginas = convert_from_path(pdf_path, dpi=300)
    except Exception as e:
        print(f"Intentando con DPI más bajo: {e}")
        try:
            paginas = convert_from_path(pdf_path, dpi=200)
        except Exception as e2:
            print(f"Intentando con DPI mínimo: {e2}")
            paginas = convert_from_path(pdf_path, dpi=150)

    # Seleccionar rango de páginas
    if pagina_fin:
        paginas = paginas[pagina_inicio-1:pagina_fin]
    else:
        paginas = paginas[pagina_inicio-1:]

    print(f"📖 Procesando {len(paginas)} páginas...")

    texto_completo = ""
    palabras_totales = 0

    # Procesar cada página
    for i, pagina in enumerate(tqdm(paginas, desc="🔍 OCR Simple")):
        pagina_num = pagina_inicio + i

        try:
            # Mejorar imagen para documentos históricos
            imagen_mejorada = pagina.convert('L')  # Escala de grises
            
            # Configuración específica para texto histórico
            config = '--psm 6 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789.,;:!?()[]{}"\'áéíóúñüÁÉÍÓÚÑÜ'

            # OCR con español y configuración optimizada
            texto_raw = pytesseract.image_to_string(imagen_mejorada, lang='spa', config=config)

            # Limpieza básica
            texto_limpio = texto_raw.strip()

            # Contar palabras
            palabras_pagina = len(texto_limpio.split())
            palabras_totales += palabras_pagina

            # Agregar al texto completo
            separador = "=" * 60
            texto_completo += f"{separador}\n"
            texto_completo += f"PÁGINA {pagina_num} ({palabras_pagina} palabras)\n"
            texto_completo += f"{separador}\n\n"
            texto_completo += texto_limpio + "\n\n"

            # Progreso cada 5 páginas para mejor seguimiento
            if (i + 1) % 5 == 0:
                print(f"📊 Progreso: {palabras_totales} palabras transcritas")

        except Exception as e:
            print(f"⚠️ Error en página {pagina_num}: {e}")
            texto_completo += f"--- ERROR EN PÁGINA {pagina_num} ---\n\n"

    # Guardar archivo
    header = f"""# TRANSCRIPCIÓN AUTOMÁTICA - DOCUMENTO HISTÓRICO

**Información del proceso:**
- Archivo fuente: {os.path.basename(pdf_path)}
- Páginas procesadas: {len(paginas)}
- Total de palabras: {palabras_totales:,}
- Fecha de transcripción: {os.popen('date').read().strip() if platform.system() != "Windows" else "Windows"}

---

"""

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(header + texto_completo)

    print(f"\n✅ ¡Transcripción completada!")
    print(f"📄 Páginas: {len(paginas)}")
    print(f"📝 Palabras: {palabras_totales:,}")
    print(f"💾 Archivo: {output_path}")

    return texto_completo

✅ Todas las librerías instaladas correctamente


## ⚙️ Paso 3: Configuración Local

Esta celda configura las rutas específicas para tu entorno Windows y verifica que:
- El archivo PDF existe
- Tesseract está instalado correctamente
- El idioma español está disponible


In [ ]:
# ==============================================================================
# CONFIGURACIÓN PARA TU ENTORNO LOCAL
# ==============================================================================

# Rutas específicas para tu proyecto
PDF_PATH = r'C:\Users\VALENTINALINARES\Downloads\transcripcion-historica\45 DER 1322.pdf'
OUTPUT_PATH = r'C:\Users\VALENTINALINARES\Downloads\transcripcion-historica\transcripcion_simple.md'

# Verificar que el archivo PDF existe
if os.path.exists(PDF_PATH):
    print(f"✅ Archivo PDF encontrado: {PDF_PATH}")
    print(f"📁 Tamaño: {os.path.getsize(PDF_PATH) / (1024*1024):.2f} MB")
else:
    print(f"❌ Archivo PDF no encontrado: {PDF_PATH}")
    print("📝 Verifica que el archivo esté en la ubicación correcta")

# Verificar configuración de Tesseract
try:
    version = pytesseract.get_tesseract_version()
    print(f"✅ Tesseract versión: {version}")
except Exception as e:
    print(f"⚠️ Error con Tesseract: {e}")
    print("💡 Asegúrate de tener Tesseract instalado y configurado correctamente")

# Verificar idiomas disponibles
try:
    langs = pytesseract.get_languages()
    if 'spa' in langs:
        print("✅ Español (spa) disponible en Tesseract")
    else:
        print("⚠️ Español no disponible, usando inglés")
except Exception as e:
    print(f"⚠️ Error verificando idiomas: {e}")

print("\n🚀 Listo para transcribir el documento histórico del siglo XIX")


✅ Archivo PDF encontrado: C:\Users\VALENTINALINARES\Downloads\vidaurre\45 DER 1322.pdf
📁 Tamaño: 71.08 MB
✅ Tesseract versión: 5.5.0.20241111
✅ Español (spa) disponible en Tesseract

🚀 Listo para transcribir el documento histórico del siglo XIX


## 🚀 Paso 4: Ejecutar Transcripción

Esta celda ejecuta la transcripción OCR del documento histórico. El proceso puede tomar varios minutos dependiendo del tamaño del PDF.


In [13]:
# ==============================================================================
# EJECUTAR TRANSCRIPCIÓN
# ==============================================================================

# Ejecutar la transcripción del documento histórico
print("🎯 Iniciando transcripción del documento del siglo XIX...")
print("⏱️ Este proceso puede tomar varios minutos dependiendo del tamaño del PDF")

# Ejecutar transcripción
texto_transcrito = transcribir_documento_simple(PDF_PATH, OUTPUT_PATH)

print("\n🎉 ¡Transcripción completada exitosamente!")
print(f"📄 El archivo se guardó en: {OUTPUT_PATH}")
print("\n💡 Consejos para revisar la transcripción:")
print("   - Revisa caracteres especiales que puedan haberse mal interpretado")
print("   - Verifica nombres propios y fechas")
print("   - Corrige palabras que puedan tener errores de OCR")


🎯 Iniciando transcripción del documento del siglo XIX...
⏱️ Este proceso puede tomar varios minutos dependiendo del tamaño del PDF
🔄 Convirtiendo PDF a imágenes...
📖 Procesando 201 páginas...


🔍 OCR Simple:   2%|▏         | 5/201 [00:11<08:49,  2.70s/it]

📊 Progreso: 307 palabras transcritas


🔍 OCR Simple:   5%|▍         | 10/201 [00:29<10:48,  3.40s/it]

📊 Progreso: 676 palabras transcritas


...

📊 Progreso: 14851 palabras transcritas


🔍 OCR Simple: 100%|█████████▉| 200/201 [10:54<00:02,  2.60s/it]

📊 Progreso: 15214 palabras transcritas


🔍 OCR Simple: 100%|██████████| 201/201 [10:56<00:00,  3.26s/it]



✅ ¡Transcripción completada!
📄 Páginas: 201
📝 Palabras: 15,272
💾 Archivo: C:\Users\VALENTINALINARES\Downloads\vidaurre\transcripcion_simple.md

🎉 ¡Transcripción completada exitosamente!
📄 El archivo se guardó en: C:\Users\VALENTINALINARES\Downloads\vidaurre\transcripcion_simple.md

💡 Consejos para revisar la transcripción:
   - Revisa caracteres especiales que puedan haberse mal interpretado
   - Verifica nombres propios y fechas
   - Corrige palabras que puedan tener errores de OCR


## 🤖 Paso 5: Limpieza con Gemini AI

Esta celda configura Gemini AI para limpiar y corregir los errores de OCR:
- Instala dependencias adicionales (google-generativeai, python-docx)
- Configura la API de Gemini
- Define función de limpieza optimizada para documentos históricos


In [13]:
# ==============================================================================
# CONFIGURACIÓN AUTOMÁTICA DE GEMINI CON MODELOS ACTUALES
# ==============================================================================

# Lista de modelos actualizados basada en tu cuenta (orden de preferencia)
MODELOS_GEMINI = [
    'models/gemini-2.5-flash',           # Más rápido, ideal para muchas páginas
    'models/gemini-2.5-pro',             # Más preciso, ideal para documentos históricos
    'models/gemini-flash-latest',         # Siempre la versión más reciente
    'models/gemini-pro-latest',           # Versión estable más reciente
    'models/gemini-2.0-flash',           # Alternativa estable
    'models/gemini-2.0-flash-001',       # Versión específica estable
    'models/gemini-2.5-flash-lite',      # Versión ligera
    'models/gemini-2.0-flash-lite'       # Alternativa ligera
]

# Función para probar modelos automáticamente
def configurar_gemini_automatico():
    """
    Prueba diferentes modelos de Gemini hasta encontrar uno que funcione
    """
    genai.configure(api_key=API_KEY)
    
    print("🔍 Probando modelos de Gemini disponibles...")
    
    for modelo in MODELOS_GEMINI:
        try:
            print(f"   Probando: {modelo}")
            model = genai.GenerativeModel(modelo)
            
            # Prueba simple para verificar que funciona
            test_response = model.generate_content("Hola, ¿funcionas?")
            
            if test_response.text:
                print(f"✅ Gemini configurado correctamente con modelo: {modelo}")
                return model, modelo
            else:
                print(f"⚠️ {modelo}: Sin respuesta")
                
        except Exception as e:
            error_msg = str(e).lower()
            if "not found" in error_msg or "not supported" in error_msg:
                print(f"❌ {modelo}: No disponible")
            elif "quota" in error_msg or "429" in error_msg:
                print(f"⚠️ {modelo}: Cuota excedida")
            else:
                print(f"❌ {modelo}: Error - {str(e)[:50]}...")
    
    # Si ningún modelo funciona, usar el modelo ya configurado del diagnóstico
    print("⚠️ Usando modelo configurado en diagnóstico anterior...")
    try:
        model = genai.GenerativeModel('models/gemini-2.5-pro-preview-03-25')
        test_response = model.generate_content("Hola, ¿funcionas?")
        if test_response.text:
            print("✅ Usando modelo del diagnóstico: models/gemini-2.5-pro-preview-03-25")
            return model, 'models/gemini-2.5-pro-preview-03-25'
    except Exception as e:
        print(f"❌ Error con modelo del diagnóstico: {e}")
    
    # Si todo falla
    print("❌ Ningún modelo de Gemini está disponible")
    print("💡 Verifica tu API key y conexión a internet")
    raise SystemExit("No se pudo configurar Gemini")

# Configurar Gemini automáticamente
try:
    model, modelo_activo = configurar_gemini_automatico()
    print(f"\n🎯 Usando modelo: {modelo_activo}")
    print("🚀 Listo para procesar el documento histórico")
except SystemExit:
    print("🚫 No se puede continuar sin Gemini")
    raise


🔍 Probando modelos de Gemini disponibles...
   Probando: models/gemini-2.5-flash
✅ Gemini configurado correctamente con modelo: models/gemini-2.5-flash

🎯 Usando modelo: models/gemini-2.5-flash
🚀 Listo para procesar el documento histórico


## 📝 Paso 6: Funciones Auxiliares

Esta celda contiene funciones adicionales para:
- Crear documentos Word profesionales
- Procesar el documento completo con Gemini
- Manejar errores y guardar progreso temporal


In [16]:
# =======================================================================
# FUNCIÓN LIMPIAR TEXTO CON GEMINI
# =======================================================================

def limpiar_texto_con_gemini(pagina, num):
    """
    Envía una página de texto OCR a Gemini para limpiarla y corregir errores.
    Maneja errores, recortes largos y reintentos automáticos.
    """
    print(f"🧠 Enviando página {num} a Gemini...")

    # Limpiar caracteres no imprimibles y limitar longitud
    pagina = re.sub(r'[^\x00-\x7F]+', ' ', pagina)
    if len(pagina) > 6000:
        pagina = pagina[:6000] + "\n\n[Texto recortado por límite de tokens]"

    prompt = (
        "Eres un asistente experto en paleografía digital. "
        "Limpia y corrige errores de OCR en el siguiente texto histórico. "
        "No cambies el contenido original ni su estructura.\n\n"
        f"TEXTO PÁGINA {num}:\n{pagina}"
    )

    for intento in range(2):  # Dos intentos
        try:
            response = model.generate_content(prompt)
            texto = response.text.strip()
            print(f"✅ Página {num} procesada.")
            return texto
        except Exception as e:
            print(f"⚠️ Error en página {num}, intento {intento+1}: {e}")
            time.sleep(2)

    return f"[Error al procesar página {num}]"

# ==============================================================================
# FUNCIÓN PARA CREAR DOCUMENTO WORD
# ==============================================================================

def crear_documento_word(contenido_md, archivo_word):
    """
    Convierte el markdown limpio a un documento Word profesional
    """
    print("📝 Creando documento Word...")

    doc = Document()

    # Configurar márgenes
    sections = doc.sections
    for section in sections:
        section.top_margin = Inches(1)
        section.bottom_margin = Inches(1)
        section.left_margin = Inches(1.25)
        section.right_margin = Inches(1.25)

    # Título principal
    titulo = doc.add_heading('DOCUMENTO HISTÓRICO - SIGLO XIX', 0)
    titulo.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Subtítulo
    subtitulo = doc.add_heading('Transcripción Digital Limpia con IA', level=1)
    subtitulo.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Línea separadora
    doc.add_paragraph("─" * 80).alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_paragraph()

    # Procesar contenido página por página
    secciones = contenido_md.split('## PÁGINA')

    for i, seccion in enumerate(secciones[1:], 1):
        # Título de página
        doc.add_heading(f'PÁGINA {i}', level=2)

        # Extraer contenido
        lineas = seccion.split('\n')
        contenido_pagina = []

        for linea in lineas[1:]:
            linea = linea.strip()
            if linea and linea != '---':
                contenido_pagina.append(linea)

        # Agregar párrafos
        texto_completo = '\n'.join(contenido_pagina)
        parrafos = texto_completo.split('\n\n')

        for parrafo in parrafos:
            parrafo = parrafo.strip()
            if parrafo and not parrafo.startswith('['):
                doc.add_paragraph(parrafo)

        # Salto de página cada 5 páginas
        if i % 5 == 0 and i < len(secciones) - 1:
            doc.add_page_break()

    # Guardar
    doc.save(archivo_word)
    print(f"✅ Word creado: {archivo_word}")

# ==============================================================================
# PROCESADOR PRINCIPAL
# ==============================================================================

def procesar_documento_completo(archivo_md_path, archivo_salida_path):
    """
    Procesa todo el documento con Gemini
    """
    print("📖 Leyendo archivo OCR...")

    # Verificar archivo
    if not os.path.exists(archivo_md_path):
        print(f"❌ Archivo no encontrado: {archivo_md_path}")
        print("🔍 Buscando en directorios cercanos...")

        # Intentar rutas alternativas
        rutas_alternativas = [
            archivo_md_path.replace('2025_1_cursos_uni', '2025_cursos_uni'),
            archivo_md_path.replace('2025_cursos_uni', '2025_1_cursos_uni'),
        ]

        for ruta in rutas_alternativas:
            if os.path.exists(ruta):
                print(f"✅ Encontrado en: {ruta}")
                archivo_md_path = ruta
                break
        else:
            # Listar archivos disponibles
            directorio = os.path.dirname(archivo_md_path)
            if os.path.exists(directorio):
                print(f"\n📁 Archivos en {directorio}:")
                for arch in os.listdir(directorio):
                    if arch.endswith('.md'):
                        print(f"  - {arch}")
            return

    # Leer archivo
    try:
        with open(archivo_md_path, 'r', encoding='utf-8') as f:
            contenido = f.read()
    except Exception as e:
        print(f"❌ Error leyendo archivo: {e}")
        return

    # Dividir por páginas (buscar separadores)
    patrones = [
        r'={60}',  # 60 signos igual
        r'-{60}',  # 60 guiones
        r'PÁGINA \d+',  # Texto "PÁGINA N"
    ]

    paginas = [contenido]
    for patron in patrones:
        nuevas = []
        for pag in paginas:
            nuevas.extend(re.split(patron, pag))
        paginas = nuevas

    # Filtrar páginas válidas
    paginas = [p.strip() for p in paginas if len(p.strip()) > 100]

    print(f"📄 Páginas detectadas: {len(paginas)}")

    # Crear documento limpio
    documento_limpio = f"""# DOCUMENTO HISTÓRICO - SIGLO XIX
## TRANSCRIPCIÓN LIMPIA Y ESTRUCTURADA

*Procesado: {time.strftime('%d/%m/%Y %H:%M')}*
*Total páginas: {len(paginas)}*

---

"""

    exitosas = 0
    errores = 0

    # Procesar cada página
    for i, pagina in enumerate(tqdm(paginas, desc="🧹 Limpiando")):
        num = i + 1

        # Limpiar con Gemini
        limpio = limpiar_texto_con_gemini(pagina, num)

        if limpio.startswith('['):
            errores += 1
        else:
            exitosas += 1

        # Agregar al documento
        documento_limpio += f"\n\n## PÁGINA {num}\n\n{limpio}\n\n---\n"

        # Ajustar pausa según tasa de error
        if errores > exitosas * 0.3:  # Si >30% errores
            time.sleep(3)
        else:
            time.sleep(1.5)

        # Guardar progreso cada 20 páginas
        if num % 20 == 0:
            temp = archivo_salida_path.replace('.md', f'_temp_{num}.md')
            try:
                with open(temp, 'w', encoding='utf-8') as f:
                    f.write(documento_limpio)
                print(f"💾 Guardado temporal: {num}/{len(paginas)} páginas")
            except Exception as e:
                print(f"⚠️ Error guardando temporal: {e}")

    # Guardar final
    try:
        with open(archivo_salida_path, 'w', encoding='utf-8') as f:
            f.write(documento_limpio)
        print(f"✅ Markdown guardado: {archivo_salida_path}")

        # Crear Word
        word = archivo_salida_path.replace('.md', '.docx')
        crear_documento_word(documento_limpio, word)

    except Exception as e:
        print(f"❌ Error guardando: {e}")
        return

    # Estadísticas
    print(f"\n📊 RESUMEN:")
    print(f"📄 Total: {len(paginas)} páginas")
    print(f"✅ Exitosas: {exitosas} ({exitosas/len(paginas)*100:.1f}%)")
    print(f"❌ Errores: {errores} ({errores/len(paginas)*100:.1f}%)")
    print(f"📁 Archivos:")
    print(f"   📄 {archivo_salida_path}")
    print(f"   📝 {word}")


## 🎯 Paso 7: Ejecutar Limpieza Final

Esta celda ejecuta el proceso completo de limpieza:
- Verifica que existe el archivo de transcripción OCR
- Procesa cada página con Gemini AI
- Genera archivo Markdown limpio y documento Word
- Muestra estadísticas del proceso


In [ ]:
# ==============================================================================
# EJECUTAR LIMPIEZA CON GEMINI
# ==============================================================================

# Rutas específicas para tu entorno local
ARCHIVO_ORIGINAL = r'C:\Users\VALENTINALINARES\Downloads\transcripcion-historica\transcripcion_simple.md'
ARCHIVO_LIMPIO = r'C:\Users\VALENTINALINARES\Downloads\transcripcion-historica\documento_limpio_gemini.md'

print("🚀 Iniciando limpieza con Gemini AI...")
print("⏱️ Tiempo estimado: 10-20 minutos")
print("💡 Este proceso corregirá errores de OCR y organizará el texto")
print()

# Verificar que existe el archivo de transcripción original
if os.path.exists(ARCHIVO_ORIGINAL):
    print(f"✅ Archivo de transcripción encontrado: {ARCHIVO_ORIGINAL}")
    print(f"📁 Tamaño: {os.path.getsize(ARCHIVO_ORIGINAL) / (1024*1024):.2f} MB")
else:
    print(f"❌ Archivo de transcripción no encontrado: {ARCHIVO_ORIGINAL}")
    print("📝 Asegúrate de haber ejecutado primero la transcripción OCR")
    print("🔍 Archivos disponibles en el directorio:")
    directorio = r'C:\Users\VALENTINALINARES\Downloads\transcripcion-historica'
    if os.path.exists(directorio):
        for arch in os.listdir(directorio):
            if arch.endswith('.md'):
                print(f"  - {arch}")

# Ejecutar procesamiento
if os.path.exists(ARCHIVO_ORIGINAL):
    procesar_documento_completo(ARCHIVO_ORIGINAL, ARCHIVO_LIMPIO)
    
    print("\n🎉 ¡Proceso de limpieza completado!")
    print("\n💡 Resultados:")
    print("   📄 Markdown limpio: documento_limpio_gemini.md")
    print("   📝 Documento Word: documento_limpio_gemini.docx")
    print("\n🔍 Revisa los archivos generados para verificar la calidad")


🚀 Iniciando limpieza con Gemini AI...
⏱️ Tiempo estimado: 10-20 minutos
💡 Este proceso corregirá errores de OCR y organizará el texto

✅ Archivo de transcripción encontrado: C:\Users\VALENTINALINARES\Downloads\vidaurre\transcripcion_simple.md
📁 Tamaño: 0.41 MB
📖 Leyendo archivo OCR...
📄 Páginas detectadas: 202


🧹 Limpiando:   0%|          | 0/202 [00:00<?, ?it/s]

🧠 Enviando página 1 a Gemini...
✅ Página 1 procesada.


🧹 Limpiando:   0%|          | 1/202 [00:15<51:30, 15.37s/it]

🧠 Enviando página 2 a Gemini...
✅ Página 2 procesada.


...

💾 Guardado temporal: 200/202 páginas
🧠 Enviando página 201 a Gemini...
✅ Página 201 procesada.


🧹 Limpiando: 100%|█████████▉| 201/202 [3:00:21<00:51, 51.16s/it]

🧠 Enviando página 202 a Gemini...
⚠️ Error en página 202, intento 1: 504 Deadline Exceeded
✅ Página 202 procesada.


🧹 Limpiando: 100%|██████████| 202/202 [3:12:04<00:00, 57.05s/it] 


✅ Markdown guardado: C:\Users\VALENTINALINARES\Downloads\vidaurre\documento_limpio_gemini.md
📝 Creando documento Word...
✅ Word creado: C:\Users\VALENTINALINARES\Downloads\vidaurre\documento_limpio_gemini.docx

📊 RESUMEN:
📄 Total: 202 páginas
✅ Exitosas: 202 (100.0%)
❌ Errores: 0 (0.0%)
📁 Archivos:
   📄 C:\Users\VALENTINALINARES\Downloads\vidaurre\documento_limpio_gemini.md
   📝 C:\Users\VALENTINALINARES\Downloads\vidaurre\documento_limpio_gemini.docx

🎉 ¡Proceso de limpieza completado!

💡 Resultados:
   📄 Markdown limpio: documento_limpio_gemini.md
   📝 Documento Word: documento_limpio_gemini.docx

🔍 Revisa los archivos generados para verificar la calidad
